### 1. Load smplxflame_30 dataset

In [13]:
import numpy as np

joint_path = "./dataloaders/utils/data_tools"
data_path = "./EMAGE/test_sequences/smplxflame_30/2_scott_0_4_4.npz"
m_data = np.load(data_path, allow_pickle=True)
print(m_data["poses"].shape)
print(m_data["trans"].shape)
print(m_data["betas"].shape)
print(m_data["expressions"].shape)
# 2004 can be different for each data(person) why? time sequence different



(2004, 165)
(2004, 3)
(300,)
(2004, 100)



### 2. Load wave16k data

In [12]:
import librosa

audio_data_path="./EMAGE/test_sequences/wave16k/2_scott_0_4_4.wav"
audio_each_file, sr = librosa.load(audio_data_path)
print(audio_each_file.shape, sr)

# resample from 22050 to 16000
audio_each_file = librosa.resample(audio_each_file, orig_sr=sr, target_sr=16000)
print(audio_each_file.shape)

(1477350,) 22050
(1072000,)


### 3. Load textgrid data

In [20]:
import textgrid as tg

vocab_path = "./EMAGE/test_sequences/weights/vocab.pkl"
word_data_path = "./EMAGE/test_sequences/textgrid/2_scott_0_4_4.TextGrid"
tgrid = tg.TextGrid.fromFile(word_data_path)
for j, word in enumerate(tgrid[0]):
    word_n, word_s, word_e = word.mark, word.minTime, word.maxTime
    print(word_n, word_s, word_e)

 0.0 0.53
my 0.53 0.93
favorite 0.93 1.34
kind 1.34 1.57
of 1.57 1.65
movies 1.65 2.2
are 2.2 2.45
romantic 2.45 3.2
movies 3.2 3.72
 3.72 3.75
such 3.75 4.1
as 4.1 4.33
titanic 4.33 5.23
 5.23 5.78
it's 5.78 6.19
a 6.19 6.46
 6.46 6.49
fantastic 6.49 7.13
film 7.13 7.56
it 7.56 7.77
captured 7.77 8.28
many 8.28 8.73
young 8.73 9.1
people's 9.1 9.44
hearts 9.44 9.79
with 9.79 10.02
it's 10.02 10.17
amazing 10.17 10.65
music 10.65 11.12
and 11.12 11.36
sentimental 11.36 11.92
plots 11.92 12.47
 12.47 12.84
when 12.84 12.98
i 12.98 13.12
think 13.12 13.28
of 13.28 13.35
the 13.35 13.42
movie 13.42 13.62
titanic 13.62 14.2
 14.2 14.23
the 14.23 14.42
word 14.42 14.92
that 14.92 15.06
comes 15.06 15.3
to 15.3 15.39
my 15.39 15.5
mind 15.5 15.91
mind 15.91 16.06
 16.06 16.41
to 16.41 16.6
mises 16.6 17.07
the 17.07 17.15
whole 17.15 17.39
film 17.39 17.94
 17.94 17.97
would 17.97 18.18
be 18.18 18.62
 18.62 19.09
love 19.09 19.94
 19.94 20.07
it's 20.07 20.27
a 20.27 20.36
kind 20.36 20.83


### npy(T,h,w,3) to mp4 video via cv2

In [ ]:

import cv2
img_sequence = np.load("out5.npy")
print(img_sequence.shape)
img_sequence = (img_sequence).astype(np.uint8)

height, width, _ = img_sequence.shape[1:4]
size = (width, height)
out = cv2.VideoWriter('pred_nobody.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 30, size)

for img in img_sequence:
    img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    out.write(img_bgr)


out.release()

print("fin")

### wav chunking

In [ ]:
import librosa
import soundfile as sf


scene01_audio_path = "../../../dataset/PXB184/scene01_audio.wav"
audio_each_file, sr = librosa.load(scene01_audio_path)

print(audio_each_file.shape, sr) 

fps = 30
num_chunks = 15
frame_per_chunk = 600
chunk_duration = frame_per_chunk / fps # 20s
samples_per_chunk = int(chunk_duration*sr) # 20*22050=

audio_chunks = [audio_each_file[i*samples_per_chunk:(i+1)*samples_per_chunk] for i in range(num_chunks)]

for i, chunk in enumerate(audio_chunks):
    print(f"chunk {i+1}: {chunk.shape}")
    start_sample = i*samples_per_chunk
    end_sample = start_sample + samples_per_chunk
    chunk = audio_each_file[start_sample:end_sample]

    output_file_name = f"scene01_audio_chunk_{i+1}.wav"
    output_path = f"../../../dataset/PXB184/{output_file_name}"

    sf.write(output_path, chunk, sr)
    print(f"saved {output_file_name}")



### video + audio

In [ ]:
def combine_audio(vidname, audname, outname, fps=30):
    import moviepy.editor as mpe
    my_clip = mpe.VideoFileClip(vidname)

    if my_clip.fps is None:
        my_clip.fps = fps

    audio_background = mpe.AudioFileClip(audname)
    final_clip = my_clip.set_audio(audio_background)
    final_fps = my_clip.fps if my_clip.fps is not None else fps

    final_clip.write_videofile(outname,fps=final_fps)

idx = 7
vidname =f"../../../../Videos/gt{idx}.mp4"
audname = f"../../../dataset/PXB184/aa/scene01_audio_chunk_{idx}.wav"
combine_audio(vidname=vidname, audname=audname, outname=f"combined{idx}.mp4",fps=30)